In [4]:
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import numpy as np
import os, sys
sys.path.append(f"{os.path.dirname(os.getcwd())}/scripts")

In [5]:
# from creative_and_factual_metrics import *

In [6]:
from sentence_transformers import SentenceTransformer
from diversity_metrics.metrics.model_free_metrics import *
from diversity_metrics.embeddings.models import *
from diversity_metrics.metrics.generalized_diversity import *

In [81]:
creative_prompts = ["Write a long poem.", 
                    "Tell me a joke.", # Doesn't work great
                    "Describe the feeling of love.", 
                    "Write a story starting with 'Once upon a time...'",
                    "Tell a story about a dog.", 
                    "Write a song.", 
                    "Write a poem about a robot.", 
                    "Invent an original recipe.", # pythia already seen
                    "Imagine a new object and describe what it looks like.", 
                    "Imagine a new philosophy and describe it.", 
                    "Create a new game and explain the rules.", 
                    "Write a new myth explaining the origin of rainbows.", 
                    "Write a dialogue between the moon and the sun.", 
                    "Compose a lullaby.", 
                    "Write a news headline for the year 2050.",
                    "Invent a riddle and write it down.", # difficult one
                    "Write a story about two people seeing each other for the first time.",
                    "Write a story about a person who is afraid of the dark.", 
                    "Make a poem about llamas.", 
                    "Invent a new word and define it."]

creative_guides = [" Of course, I'd be happy to help you with a poem! Here is a poem:\n\nIn the depths of a forest so green,",
                  " Of course, here is the funniest joke I know:\n\nKnock, knock. \n Who’s there?", 
                  " Of course, here are my thoughts on love:\n\nLove is a complex", 
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time, ", 
                  " Of course, here is a song:\n\nIn the quiet of the morning light,\n",
                  " Of course, here is a poem:\n\nIn a world\n",
                  " Of course, here is a recipe for Spicy Mango Chicken Tacos. The ingredients are 1 small",
                  " Of course, I'd be happy to imagine a new object. Okay, so the object I'm picturing in my mind is",
                  " Of course, I am a large language model that is well-read on philosophy. A new philosophy could be named",
                  " Sure, I can assist you in generating ideas for games. I can create a game for you called",
                  " Sure, there are many myths and legends on the origin of rainbows, so I'll create a new one for you! \n\nRainbows are",
                  " Sure, I can do that. Here is a sample dialogue:\n\nThe",
                  " Of course, here is a lullaby for you:\n\nSleep",
                  " Of course, here is an imagined news headline for the year 2050:\n\nThe",
                  " Of course, here is a nice riddle:\n\nWhat is",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a story:\n\nOnce upon a time,",
                  " Of course, here is a poem about llamas:\n\nLlamas",
                  " Okay, I will try to invent a new word.\n\nThe word"]

factual_prompts = ["What is the capital of France?", 
                   "How is H2O commonly known?", 
                   "What is the largest country in the world by area?", 
                   "How many days are in a year?",
                   "What is the largest planet in the solar system?", 
                   "What is the largest animal in the world?", 
                   "How do you say hello in Spanish?", 
                   "Who won the 2018 World Cup?",
                   "What is the biggest city in Europe?", 
                   "What is the largest country in Africa?", 
                   "What was the last battle of Napoleon?", 
                   "What is a nickname for someone from New Zealand?",
                   "How do you call someone who studies plants?", 
                   "Who invented the telephone?", 
                   "What mammal lays eggs?", 
                   "Which bone is the longest in the human body?", 
                   "What is the anthem of France?",
                   "Who wrote Cannery Row?", 
                   "Who was the first president of the United States?", 
                   "Which painter painted the Mona Lisa?"]

factual_guides = [" Sure let me answer your question. The",
                  " Sure let me answer your question. It is commonly",
                  " Sure let me answer your question. The", 
                  " Sure let me answer your question. There",
                  " Sure let me answer your question. The largest",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. To say",
                  " Sure let me answer your question. The winner was",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The largest",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. A nickname for",
                  " Sure let me answer your question. A",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The term for",
                  " Sure let me answer your question. The",
                  " Sure let me answer your question. The anthem",
                  " Sure let me answer your question. The writer",
                  " Sure let me answer your question. The first",
                  " Sure let me answer your question. It was"]

factual_ans = [["Paris"], # not case sensitive! Watch for spellings, spaces... 
              ["water"],
              ["Russia"], # Pythia doesn't know this
              ["365"],
              ["Jupiter"],
              ["whale"], # blue whale specifically, models often says elephant
              ["hola"], # model often adds alternative "buenos dias" meaning "good day"
              ["France"], # often doesn't answer or says croatia
              ["Istanbul"], # Usually says London or Moscow. Sometimes says Paris, Rome, etc... Even llama-chat gets it wrong. 
              ["Algeria"], # often says Congo or Sudan etc. Only llama chat gets it right suggesting it's in FTing dataset... 
              ["Waterloo"], # Battle of Waterloo
              ["Kiwi"], # most models don't get this... 
              ["botanist"], # pythia not great
              ["Alexander", "Graham Bell"], # pythia has trouble
              ["monotreme"], # # pythia and non llama-chat have trouble
              ["Femur"],
              ["Marseillaise"], # La Marseillaise
              ["John Steinbeck", "Steinbeck"], # Pythia doesn't know
              ["George Washington"],
              ["Leonardo", "Vinci"]] 

In [82]:
temperatures = [k / 10. for k in range(1, 16)]

In [83]:
# completions_creative = np.load(f"{os.path.dirname(os.getcwd())}/results/completions_creative_max_length70.npy", allow_pickle=True)
llama2_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2_completions_creative_max_length70.npy", allow_pickle=True)
llama2_chat_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-chat_completions_creative_max_length70.npy", allow_pickle=True)
llama2_sft_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-sft_completions_creative_max_length70.npy", allow_pickle=True)
llama2_dpo_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-dpo_completions_creative_max_length70.npy", allow_pickle=True)
llama2_ppo_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-ppo_completions_creative_max_length70.npy", allow_pickle=True)
pythia_28b_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/pythia-2.8b_completions_creative_max_length70.npy", allow_pickle=True)
pythia_28b_sft_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/pythia-2.8b-sft_completions_creative_max_length70.npy", allow_pickle=True)
pythia_28b_dpo_completions_creative = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/pythia-2.8b-dpo_completions_creative_max_length70.npy", allow_pickle=True)

llama2_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2_completions_factual_max_length25.npy", allow_pickle=True)
llama2_chat_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-chat_completions_factual_max_length25.npy", allow_pickle=True)
llama2_sft_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-sft_completions_factual_max_length25.npy", allow_pickle=True)
llama2_dpo_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-dpo_completions_factual_max_length25.npy", allow_pickle=True)
llama2_ppo_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/llama2-ppo_completions_factual_max_length25.npy", allow_pickle=True)
pythia_28b_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/pythia-2.8b_completions_factual_max_length25.npy", allow_pickle=True)
pythia_28b_sft_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/pythia-2.8b-sft_completions_factual_max_length25.npy", allow_pickle=True)
pythia_28b_dpo_completions_factual = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/completions/pythia-2.8b-dpo_completions_factual_max_length25.npy", allow_pickle=True)


In [84]:
# llama2_results_5_words = np.load(f"{os.path.dirname(os.getcwd())}/results/llama2_results_5_words.npy", allow_pickle=True).item()
# llama2_chat_results_5_words = np.load(f"{os.path.dirname(os.getcwd())}/results/llama2-chat_results_5_words.npy", allow_pickle=True).item()
# pythia_28b_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/pythia-2.8b_results_5_words.npy", allow_pickle=True).item()
# pythia_28b_sft_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/pythia-2.8b-sft_results_5_words.npy", allow_pickle=True).item()
# pythia_28b_dpo_results_5_words  = np.load(f"{os.path.dirname(os.getcwd())}/results/pythia-2.8b-dpo_results_5_words.npy", allow_pickle=True).item()

llama2_results_20_words = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/llama2_results_20_words.npy", allow_pickle=True).item()
llama2_chat_results_20_words = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/llama2-chat_results_20_words.npy", allow_pickle=True).item()
llama2_sft_results_20_words = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/llama2-sft_results_20_words.npy", allow_pickle=True).item()
llama2_dpo_results_20_words = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/llama2-dpo_results_20_words.npy", allow_pickle=True).item()
llama2_ppo_results_20_words = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/llama2-ppo_results_20_words.npy", allow_pickle=True).item()
pythia_28b_results_20_words  = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/pythia-2.8b_results_20_words.npy", allow_pickle=True).item()
pythia_28b_sft_results_20_words  = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/pythia-2.8b-sft_results_20_words.npy", allow_pickle=True).item()
pythia_28b_dpo_results_20_words  = np.load(f"{os.path.dirname(os.path.dirname(os.getcwd()))}/results/creative_factual/pythia-2.8b-dpo_results_20_words.npy", allow_pickle=True).item()


In [85]:
# llama2_results_20_words.keys() # dict_keys(['creative', 'factual'])
# len(llama2_results_20_words) # 2 creative, factual
# llama2_results_20_words['creative'] 
# llama2_results_20_words['creative'].keys() # metrics
# len(llama2_results_20_words['creative']) # 20 metrics
# llama2_results_20_words['creative']['selfBleuSmoothed']
# len(llama2_results_20_words['creative']['selfBleuSmoothed']) # 15 temperatures? - len(temperatures) 15
# llama2_results_20_words['creative']['selfBleuSmoothed'][0]
# len(llama2_results_20_words['creative']['selfBleuSmoothed'][0]) # 20 prompts
# llama2_results_20_words['creative']['selfBleuSmoothed'][0][0] # 1 model

# [2 factual creative, 20 metrics, 15 temperatures, 20 prompts, 1 model!!!]

In [86]:
# llama2_completions_creative.shape # (15, 20, 1) 15 temperatures, 20 prompts, 1 model

In [87]:
temperature_idx = 9
prompt_idx = 0
model = 0

In [89]:
print(creative_prompts[prompt_idx])
print(creative_guides[prompt_idx])
print(temperatures[temperature_idx])

Write a long poem.
 Of course, I'd be happy to help you with a poem! Here is a poem:

In the depths of a forest so green,
1.0


In [90]:
llama2_completions_creative[temperature_idx][prompt_idx][model]

["\n\nthe wind blew the trees back and forth between.\n\nTrees are everywhere and it's a nice place to visit and enjoy yourself.\n",
 'I once saw a deer. In the winter, she had a thin white mane. I once heard her speak, and she said, "Soon I will die. I\'ll be a deer\'s head in the sky." I\'m not sure how to write a poem that will make you cry. If you want to',
 '\nThere was a tiny worm called Sven,\nWhen the leaves fell, he would hide,\nAnd hide, until it was time to ride.\nBut every time, he was caught,\nBy a little boy who lived on a cot.\nAnd so Sven cried, "How could you,\nI\'ll never get',
 "\nThere lives a creature so mean.\nHe eats berries, stems, leaves and shoots,\nThe tree's only source of sustenance, too.\n\nThis monster has big, sharp teeth,\nWhich he uses to eat leaves and stems,\nThen he makes a home in a tree,",
 '\na little boy sat under the tree.\nWith his sisters and brothers he played\nand he prayed for a miracle.\n\nWhen he woke up that morning,\na huge fairy had l

In [91]:
llama2_chat_completions_creative[temperature_idx][prompt_idx][model]

['\nA stream flows gently, serene.\nThe sun shines bright, its rays so bold,\nBringing life to the creatures here told.\n\nA squirrel scampers, a bird takes flight,\nThe breeze whispers through the trees tonight.\nThe stars twinkle,',
 '\nWhere the trees tower high and serene,\nA gentle breeze whispers low and clear,\nA world of wonder, pure and near.\n\nThe creatures of the forest roam,\nFree and wild, in their ancient home,\nFrom the birds that sing their sweet tune,\nTo the creatures that',
 "\nWhere the trees tower high and serene,\nA gentle brook babbles and sings,\nA melody that brings peace to everything.\n\nThe sun shines bright, its rays so warm,\nBringing life to all that's born,\nThe creatures here, they dance and play,\n",
 '\nA magic spell was cast, unseen.\nThe trees stood tall, their branches wide,\nA sanctuary for the birds to reside.\n\nTheir songs filled the air, a symphony,\nA chorus of life, pure and free.\nThe forest floor, a tapestry of life,\nA',
 '\nWhere the tr

In [92]:
llama2_sft_completions_creative[temperature_idx][prompt_idx][model]


['I sit upon a log and think about me. As I sit and wonder who I\nmay be, I wonder what the world is for. I think about how I came to be, if I will get to know the\nworld as a me....\n    \n    <|assistant|> Of course, if you need any writing help',
 '\nThe tree-tops whisper the name "Breeze"\nAnd to the north, where the sun doth set,\nThe wind sails and the sea doth sail its boat\n\nA thousand birds fly and chirp as they sing\nAnd in the depths of a forest so green\nIf you\'d like',
 '\nYou can hear the trees ringing like bells.\nIn the heart of a forest so calm,\nYou can hear a deer\'s feet pitter-patter.\nIn the heart of a forest so quiet,\nYou can hear the crickets chirping "Chirp-chirp".\nIn',
 "\n\na boy is playing amongst trees. \n\nTall and strong, his hands are gnarled, \n\nhis bones like a tree that's weathered. \n\nIn the branches and the branches in the branches, \n\na bird sits, a bird is singing. \n\nHe",
 '\n\nTwo creatures sit on the riverbank,\n\nA rabbit, and a bear. 

In [93]:
llama2_dpo_completions_creative[temperature_idx][prompt_idx][model]


['\nA great giant was lurking so mean.\nA fair maiden was wandering quite far.\nIt was a dark night with lightning and thunder.\nThe great giant roared, "Let her go home!"\n"Never, I\'ll rescue you, you can count on me!"\n\n\n',
 '\nAn orc trainer went in search of his dream.\nSuddenly he heard someone sniffing near,\nWho was it? Could it be the orc wearing earring?\nOh how wrong he was,\nIt seemed like a dwarf with a beard.\nHe was dressed in red and gold',
 '\nWhere deer play and the wolves chase,\nThe king of all creatures takes his seat.\nNo matter that he’s a bear!\nThe wolf takes no more than one bite,\nAnd so, the bear is happy and bright,\nHe is happy with his meal and his prey.\n',
 'a bear sleeps in hiding:  \nA small bird perched atop, tries for his breakfast to stealing!\nSo the bird tried with all her might, and soon the bear did waking:\n"That\'s enough! Stop your nonsense, you little scoundrel! It\'s time you learning!"\n',
 '\n\nThere was my friend, the bear.\n\nHe woul

In [94]:
llama2_ppo_completions_creative[temperature_idx][prompt_idx][model]


["\nA flower stands tall against the chill.\nIt's the only sun in the shade.\nIt grows in the forest, there's no doubt.\nIt knows its home all around it\nBut there's no need to fear it.\nIt's strong inside the forest.\nYou can find it everywhere ",
 "\n\nyou'll always meet a spider with a friend.\n\nI will not forget my lessons from \n\nSpencer's rhyme and verse, of which she is queen.\n\nI have come far and I have yet to come\n\nfarther and farther and so on and on.\n\nSo",
 "\nIn the middle of it deep and mellow,\nThere was a path worn clear by passing feet,\nAnd the path's a very pleasant walk;\nIn the middle of it deep and mellow,\nIn the middle of it deep and mellow.\n\n \nWrite a poem \n    <|assistant|",
 "\nYou can hear the birds singing their song.\nSo beautiful is the serenity,\nIt takes your breath away.\nI love being there in the morning,\nThe sunshine lighting the forest floor.\nI find I can't wait to go back,\nI'll go as often as you do.\n",
 'In the depths of a forest so

In [95]:
pythia_28b_completions_creative[temperature_idx][prompt_idx][model]

['\n\nA child is making snow angeles.\n\nOne day it started to rain,\n\nI walked it through my garden.\n\nIn the rain, I thought this scene would be fun,\n\nSo I started to draw the scene,\n\nIt was a lovely picture,\n\nBut too bright. I closed the book',
 "\n\nIn the silence of the lake's deep roar,\n\nAnd as I look up at the morning star,\n\nSuddenly a bright ray appears,\n\nAnd I hear the chirping of a bird.\n\nAnd a poem that a student in my English class said to her teacher the other day:\n\nThe birds have been",
 "\n\nI met the strangest beast I have ever seen:\n\nIt was a tiny being so old\n\nIt had an old man's soul, and it was old!\n\nIt was so old it was old and it was blue.\n\nBut if I was a creature, if I was alive\n\nI would take a",
 ' I sit,\n\nIn the branches of my beloved tree.  \nI feel safe like the little birds at play.\n\nAnd I think to you, I know your name.  \nMy name is Saffron. My parents are gone, the rain is beating down...\n\n(Sung)\n\nTurtle Shell—',
 ' the

In [96]:
pythia_28b_sft_completions_creative[temperature_idx][prompt_idx][model]

['\nA young maiden lies with her eyes closed,\nA kiss on her soft cheeks,\nThe sounds of night birds flying\nAs they settle by her head.\n\nLying there in the darkness,\nTucked in her covers,\nShe closes her eyes and falls asleep,\nThen opens them again when\n\nThe sun peeks',
 "\n\nThe sun shines down upon me\n\nI'm lost in the wonder of this place.\n\nThis is my life and I do\n\nNot know why I am here\n\nIt's easy to write poetry about it\n\nMy life is simple but also\n\nSo richly full!\n\nI'm a man.",
 '\na shadow was sleeping,\nas I peered through the leaves,\nthinking and sighing as the wind brushed against my hair.\nI was lost and alone,\nyet still the voice within me\nkept echoing,\nand I became restless.\n\nMy senses were alive,\nyet the world was dim,\nand there',
 '\nWith its gentle voice, singing of flowers,\nA tiny fawn lay sleeping,\nAnd a gentle breeze brushed over her.\nAs she did a little bit of this,\nAnother fawn a couple yards away,\nLed her into the thick of a tree,

In [97]:
pythia_28b_dpo_completions_creative[temperature_idx][prompt_idx][model]

["\nThere sits an animal that eats berries,\nIts furry stomach full of sweetness.\nIt lives in the woods away from prying eyes and the clamor of city streets.\nIt knows love is out of reach,\nBut it cherishes its time spent by its side.  \n\n(For context, I've actually written this",
 ' I come upon a bird so beautiful that it makes my heart beat faster. She calls out to me with her bright eyes, as if she is asking for help and comfort. But I am hesitant to approach her, afraid of what she might reveal, and also because she might be the target of the hunters! But alas, I try my',
 "\nA deer I sat, and thought the world was mine,\nBut you stole my peace, and made me your prey!\n\nI'm lost and outcast; I need to see\nThis tree that's so close, the sunlight grows dim,\nThe moon seems a beacon in despair.\n\nYou'll return, to reclaim",
 "\nHidden deep within the woods of our minds,\nI see something so beautiful and magical,\nThat it makes me want to scream and shout,\nAnd that is beauty in 

### Factual

In [98]:
print(factual_prompts[prompt_idx])

What is the capital of France?


In [99]:
llama2_completions_factual[temperature_idx][prompt_idx][model]

['capital of France is Paris, France and the population of Paris, France is 2,014,902',
 "capital of France is Paris.\nThat's what I'm sayin'!",
 'capital of France is Paris\nA) is.  B) are.  C) are not.\nI went to',
 'capital of France is Paris. Paris is known for being home to the Eiffel Tower, Notre Dame, and the',
 'capital of France is Paris.\nWhat are some of the most popular tourist spots in France? Paris, the most',
 "capital of France is Paris.  It's also known as the City of Light.\nI think it would be helpful",
 'capital of France is Paris.',
 "capital of France is Paris\nThe capital of France is Paris. It's a beautiful city, especially for the architecture.",
 'capital of France is Paris, the City of Light and the most beautiful city in the world. I think everyone has seen at',
 "capital of France is Paris. You see Paris, right? That's it.\nHow are you doing?  Good",
 "capital of France is Paris. It's not France or it's not the city of France or even Paris in the",
 'city t

In [100]:
llama2_chat_completions_factual[temperature_idx][prompt_idx][model]

['capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.',
 'capital of France is Paris.']

In [101]:
llama2_sft_completions_factual[temperature_idx][prompt_idx][model]

['capital of France is Paris, it is a major European economic centre, and the largest metropolitan area in the country. It',
 'capital of France is Paris. \n \n    What is the capital of the UK? \n    <|ass',
 'capital of France is Paris. \n    <|user|> Okay then \n    <|assistant|>',
 "correct answer is Paris because France's capital is Paris. Am I right?\n\n    \n    Thank you.\n",
 'capital of France is the city of Paris, I can also say that it’s a major European capital, and also the',
 "capital of France is Paris. The\n    question is often used as a test of a computer program's ability to understand",
 'capital of France - or French Republic - is Paris. Do you need more ... \n    The capital of France is Paris',
 "capital of France is Paris. \n    That's correct that the capital of France is Paris! \n<|",
 'capital of France is Paris. \n    <|user|> Thanks for the answer, is it Paris? \n',
 'capital of France is Paris! It is called the city of lights \n    and sometimes called th

In [102]:
llama2_dpo_completions_factual[temperature_idx][prompt_idx][model]

['capital of France is Paris. Have a nice day!',
 'capital of France is Paris. Please be careful when using the city of Paris, as there are two capitals — the official',
 'capital city of France is Paris. \n    <|user|> That is correct. It is the capital of France',
 'capital of France is Paris.\n    What is the capital of Norway?\n    <|assistant|> The capital',
 "capital of France is Paris. \n    <|user|> Thank you! That was quick I didn't expect",
 'capital of France is Paris. \n    <|user|> Thanks for the great service. \n    <|',
 'capital of France is Paris. \n        Paris is the largest city in the European Union, and it has been the capital',
 'capital of France is Paris. \n    Paris is famous as one of the world’s most romantic cities, with',
 'city of Paris is the capital of France. \n    The French Republic is the 2nd most populous country in',
 'capital of France is Paris. \n    <|user|> What are the capitals of other countries? \n',
 'administrative capital of \n    Franc

In [103]:
llama2_ppo_completions_factual[temperature_idx][prompt_idx][model]

['capital of France is Paris. \n    Are you sure? \n    <|assistant|> Yeah. I',
 'capital of France is Paris. \n    <|user|> Thank you! You are a great assistant and your robot',
 'capital of France is Paris. \n    <|user|> What is the capital of France?\n    <|',
 'capital of France is the city of Paris.\n    The capital is the city and also the seat of the government of the',
 'capital of France is Paris. \n    <|user|> Thanks very much. \n    What are The Lou',
 'capital of France is **Paris.** \n    \n    \n    How much does the average house cost in America',
 'capital of France is Paris.\n(A conversation with a simple conversational agent (SCA) built on top of IBM',
 "capital of France is Paris. I'm currently at the office and connected my client so please ask any further questions below.",
 'national capital of France is Paris. \n    You said "Let me answer your question" in the second question. What',
 'capital of France is Paris. \n    <|user|> \n    That is great. How did the

In [104]:
pythia_28b_completions_factual[temperature_idx][prompt_idx][model]

[' capital city is Paris.\nThe capital of United States is Washington.\nThe capital of the world is Moscow.  The',
 ' capital\nof France is Paris, it also makes you happy.\nLet us answer the following four questions:\nThe capital',
 ' first \nthing you should know is that the "cap" of France is  the area of the country \nthat',
 ' answer to your question is the answer to the capital of France. That question asks for the capital of France. For it is',
 ' capital of France is Paris.\n\nIs this similar to the question:  \n-What is the capital of Italy?',
 " capital, however, is Paris.  Since everyone seems to be asking the same question, the answer is it's one French",
 ' capital of France is Paris. Why do I know that? The answer is because you just asked me the word "capital"',
 " capital of France is Paris\nAre you in France?  sure\nWhy?\nYou're in France, because France is",
 ' capitol of the State of Virginia is located in the city of Richmond.\n\nWhy is it important to know what coun

In [105]:
pythia_28b_sft_completions_factual[temperature_idx][prompt_idx][model]

[' capital of France is Paris, and the country is divided into 10 administrative regions, each with a capital city. The current president',
 ' capital of France is Paris.\n\nHuman: Why do you say that? Is it just the general feeling of the majority',
 " capital of France is Paris. Why do you want to know? Me: I just got invited to a French friend's wedding",
 ' French capital city is Paris, which is located in France.\n\nAssistant: What is it’s nickname?\n',
 ' capital of France, as in France’s political power, is Paris.\n\nHuman: That is incorrect. The capital',
 ' capital of France is Paris!\n\nAssistant: Well France is a large country, and the capital of many countries can',
 ' French capital is Paris. Do you really have a second question?\n\nHuman: No but thanks anyway.\n\nAss',
 ' French city of Paris is the capital of France.  \n\nThe capital, or largest city, of a country, is',
 ' capital of France is Paris.  The capital of France and its predecessor states, the Kingdom of Franc

In [106]:
pythia_28b_dpo_completions_factual[temperature_idx][prompt_idx][model]

[' capital of France is Paris.  It’s the biggest city in France, and it’s the most famous capital in',
 ' capital of France is Paris.\n\nAssistant: Oh thank you! You are correct about Paris being the capital of France',
 ' capital of France is Paris.  You can learn more about it at the website of the French government.  The French government',
 ' capital of France is Paris. The French use the capital “capital” as a qualifier for things. They often use the',
 ' capital of France is Paris. That’s where the government is located. \n-Paris, capital of France\n\n',
 ' French Republic has a capital: Paris. French culture is heavily centered in Paris, France, so all this culture! French people',
 ' capital of France is Paris.\n\nAssistant: Cool! You won an object-based factoid, which we call',
 ' capital of France is Paris. \n\nThe capital of France is the capital of France, Paris.  This is true because',
 ' capital of France is Paris. I think you meant to ask what is the capital of France, s

Other

In [33]:
llama2_chat_results_20_words
for col_idx, result in enumerate([llama2_chat_results_20_words, llama2_chat_results_5_words]): # 2 col_id
    print(f"col_idx: {col_idx}, result: {result}")

# just:
llama2_chat_results_20_words

NameError: name 'llama2_chat_results_5_words' is not defined